In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

L0 = Dense(units=1, input_shape = [1])
model = Sequential([L0])
model.compile(optimizer='sgd', loss='mean_squared_error')

xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

model.fit(xs, ys, epochs = 500)
print(model.predict([10.0]))
print("Here is what i've learned: {}".format(L0.get_weights()))

7us/step - loss: 0.0028
Epoch 264/500
1/1 [==============================] - 0s 2ms/step - loss: 0.0028
Epoch 265/500
1/1 [==============================] - 0s 2ms/step - loss: 0.0027
Epoch 266/500
1/1 [==============================] - 0s 2ms/step - loss: 0.0027
Epoch 267/500
1/1 [==============================] - 0s 2ms/step - loss: 0.0026
Epoch 268/500
1/1 [==============================] - 0s 2ms/step - loss: 0.0025
Epoch 269/500
1/1 [==============================] - 0s 2ms/step - loss: 0.0025
Epoch 270/500
1/1 [==============================] - 0s 998us/step - loss: 0.0024
Epoch 271/500
1/1 [==============================] - 0s 3ms/step - loss: 0.0024
Epoch 272/500
1/1 [==============================] - 0s 998us/step - loss: 0.0023
Epoch 273/500
1/1 [==============================] - 0s 2ms/step - loss: 0.0023
Epoch 274/500
1/1 [==============================] - 0s 2ms/step - loss: 0.0022
Epoch 275/500
1/1 [==============================] - 0s 2ms/step - loss: 0.0022
Epoch 276/50

In [2]:
export_dir = 'saved_model/1'
tf.saved_model.save(model, export_dir)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved_model/1\assets


In [3]:
#Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

In [4]:
import pathlib
tflite_model_file = pathlib.Path('model.tflite')
tflite_model_file.write_bytes(tflite_model)

764

In [5]:
#Load tflite model and allocate the tensors
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

#Get the input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

[{'name': 'dense_input', 'index': 0, 'shape': array([1, 1]), 'shape_signature': array([-1,  1]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'Identity', 'index': 3, 'shape': array([1, 1]), 'shape_signature': array([-1,  1]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [ ]:
to_predict = np.array([[10.0]], dtype = np.float32)
print(to_predict)
interpreter.set_tensor(input_details[0]['index'], to_predict)
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])
print(tflite_results)